In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import seaborn as sns

In [2]:
plt.rcParams["figure.figsize"]=[15,5]

In [3]:
raw_pp=pd.read_csv('9thhour.csv', usecols=['timestamp', 'data.ax', 'data.az'])
raw_pp.sort_values(by=['timestamp'], inplace=True)
raw_pp.reset_index(inplace=True)
raw_pp.dropna(axis=0, inplace=True)
raw_pp['timestamp'] =  pd.to_datetime(raw_pp['timestamp'])

In [4]:
#Load LD data and plot
raw_pp["acc"] = ( raw_pp["data.ax"]**2 + raw_pp["data.az"]**2) ** 0.5
acc=raw_pp[["timestamp", "acc"]]
test_pp=acc
# test_pp['acc_rolling_sum']=pd.Series.to_frame(test_pp.acc.rolling(100*10, center=True).sum())
test_pp['acc_rolling_std']=pd.Series.to_frame(test_pp.acc.rolling(100*2, center=True).std())


/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
%matplotlib qt

In [6]:
# plt.plot(test_pp['timestamp'], test_pp['acc'])
# plt.plot(test_pp['timestamp'], test_pp['acc_rolling_std'])

In [7]:
test_pp['detections']=pd.Series.to_frame(test_pp.acc_rolling_std.rolling(100, center=True).sum())

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
# plt.plot(test_pp['timestamp'], test_pp['acc']*100)
# plt.plot(test_pp['timestamp'], test_pp['detections'])

In [9]:
# plt.plot(test_pp['timestamp'], test_pp['detections'])

In [10]:
test_pp['boards']=0
test_pp['boards'][test_pp['detections']>=2]=1

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
# plt.plot(test_pp["timestamp"],test_pp["boards"])

In [12]:
working_times_raw=scipy.signal.find_peaks(test_pp.boards, distance=40*72, width=1)

In [13]:
working_times_raw_df=pd.DataFrame({"sample_number":working_times_raw[0], "working_time":working_times_raw[1]['widths']/72})

In [14]:
test_pp['boards']=test_pp['boards']*-1
idle_times_raw=scipy.signal.find_peaks(test_pp.boards, distance=40*72, width=1)
idle_times_raw_df=pd.DataFrame({"sample_number":idle_times_raw[0], "working_time":idle_times_raw[1]['widths']/72})
test_pp['boards']=test_pp['boards']*-1

In [15]:
for x, row in working_times_raw_df.iterrows():
    working_times_raw_df.ix[x,'start_time']=test_pp.iloc[int(working_times_raw[1]['left_ips'][x])].timestamp
    working_times_raw_df.ix[x,'end_time']=test_pp.iloc[int(working_times_raw[1]['right_ips'][x])].timestamp
    working_times_raw_df.ix[x,'timestamp']=test_pp.iloc[int(working_times_raw[0][x])].timestamp

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after remo

In [16]:
for x, row in idle_times_raw_df.iterrows():
    idle_times_raw_df.ix[x,'start_time']=test_pp.iloc[int(idle_times_raw[1]['left_ips'][x])].timestamp
    idle_times_raw_df.ix[x,'end_time']=test_pp.iloc[int(idle_times_raw[1]['right_ips'][x])].timestamp
    idle_times_raw_df.ix[x,'timestamp']=test_pp.iloc[int(idle_times_raw[0][x])].timestamp

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after remo

In [17]:
working_times_df=working_times_raw_df[working_times_raw_df.working_time>30]

In [18]:
working_times_df['event']=1
idle_times_raw_df['event']=0

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
raw_pp_current=pd.read_csv('9thhourcur.csv', usecols=['timestamp', 'data.A1', 'data.A2', 'data.A3'])
raw_pp_current.sort_values(by=['timestamp'])
raw_pp_current.reset_index(inplace=True)
raw_pp_current['total_current']=raw_pp_current['data.A1']+raw_pp_current['data.A2']+raw_pp_current['data.A3']
raw_pp_current['timestamp']=pd.to_datetime(raw_pp_current['timestamp'])
raw_pp_current.index=raw_pp_current['timestamp']


In [20]:
# working_times_df['energy']=raw_pp_current[(raw_pp_current['timestamp'] <= working_times_df['end_time']) & (raw_pp_current['timestamp'] >= working_times_df['start_time'])]['total_current'].sum()*230

In [21]:
working_times_df['energy'] = working_times_df.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.end_time) & 
                                                            (x.start_time <= raw_pp_current.timestamp),
                                                            ['total_current']].sum()*230/3600000, axis=1)

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
idle_times_raw_df['energy'] = idle_times_raw_df.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.end_time) & 
                                                            (x.start_time <= raw_pp_current.timestamp),
                                                            ['total_current']].sum()*230/3600000, axis=1)

In [23]:
working_times_df.index=working_times_df.timestamp
working_times_df.drop('timestamp', axis=1, inplace=True)
working_times_df.drop('sample_number', axis=1, inplace=True)

/home/raghu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [24]:
idle_times_raw_df.index=idle_times_raw_df.timestamp
idle_times_raw_df.drop('timestamp', axis=1, inplace=True)
idle_times_raw_df.drop('sample_number', axis=1, inplace=True)

In [25]:
# plt.plot(test_pp["timestamp"],test_pp["boards"])
# plt.plot(raw_pp_current.timestamp, raw_pp_current['total_current'])

In [26]:
PP_events=working_times_df[['event', 'working_time', 'energy']]

In [27]:
idle_times_df=idle_times_raw_df[['event', 'working_time', 'energy']]
PP_events=PP_events.append(idle_times_df)

In [28]:
sns.distplot(PP_events.working_time)
plt.title('PP2 Working Times Histogram')

/home/raghu/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Text(0.5, 1.0, 'PP2 Working Times Histogram')

In [29]:
print('PP working time mode: ', PP_events.working_time.mode().mean())

PP working time mode:  59.111111111111114


In [30]:
PP_events.loc[PP_events.event==1, 'energy'].mean()

0.010297117424242426

In [31]:
PP_events.loc[PP_events.event==0, 'energy'].mean()

0.006257277777777777

In [36]:
plt.plot(test_pp.timestamp, test_pp.acc)

In [37]:
plt.plot(test_pp.timestamp, test_pp.boards)

In [38]:
PP_events[PP_events.event==0]

,event,working_time,energy
timestamp,,,
2019-01-02 09:04:45.123017,0,55.777778,0.004700
2019-01-02 09:07:27.439327,0,155.277778,0.012992
2019-01-02 09:10:08.525213,0,53.458333,0.004479
2019-01-02 09:11:38.770379,0,6.750000,0.000447
2019-01-02 09:12:19.104090,0,14.430556,0.001234
2019-01-02 09:13:53.142894,0,49.916667,0.004131
2019-01-02 09:15:45.521730,0,47.555556,0.003902
2019-01-02 09:17:10.849731,0,20.069444,0.001450
2019-01-02 09:17:58.228741,0,6.791667,0.000561
